# A' specrum

The models consists of a dark photon ($A'$) with mass $m_{A'}$, and two nearly degenerate states in the dark sector ($\chi_1,~\chi_2$), with $m_2>m_1$ and $\Delta=\frac{m_2-m_1}{m_1}$. The dark photon - dark sector interaction has an off-diagonal coupling and the dark photon is as usual. We have,


\begin{equation}
\mathcal{L} = \left(i\epsilon_D A'_{\mu}\bar{\chi_2}\gamma^{\mu}\chi_1 + h.c\right) + \frac{1}{2} m_{A'}^2 A'^2  - \epsilon e \sum \bar f \gamma^\mu f A'_\mu.
\end{equation}

We set $\alpha_D = \epsilon_D^2/(4\pi)$ large so that BR($A'\rightarrow \chi_1 \chi_2) \approx 1$.


We first require the A' spectrum. We follow the darkphoton notebook for this part. This part need be run only once.

## Load Libraries 

In [12]:
import sys, os
import random, math
src_path = "../../"
sys.path.append(src_path)

import numpy as np
from src.foresee import Foresee, Utility, Model
from matplotlib import pyplot as plt

## Specify the model

In [13]:
energy = "13.6"
modelname="DarkPhoton"
model = Model(modelname, path="./")

### **A' Production:** 

If the dark photon is sufficiently light, it is primarily produced in the decay of pseudoscalar mesons $\pi^0$, $\eta$ and $\eta' \to \gamma A'$. The branching fractions for the leading channels are 

\begin{equation}
    \text{BR}(\pi^0 \to A' \gamma) = 2 \epsilon^2 \times\text{BR}(\pi^0 \to \gamma\gamma) \times \left(1-m_{A'}^2/m_\pi^2\right)^3
    \quad\quad\text{and}\quad\quad
    \text{BR}(\eta \to A' \gamma) = 2 \epsilon^2 \times\text{BR}(\eta \to \gamma\gamma) \times \left(1-m_{A'}^2/m_\eta^2\right)^3
\end{equation}

In the following, we model the production using `EPOSLHC`, `SIBYLL`, `QGSJET` and  `Pythia8-Forward`. 

In [15]:
model.add_production_2bodydecay(
    pid0 = "111",
    pid1 = "22",
    br = "2.*0.99 * coupling**2 * pow(1.-pow(mass/self.masses('pid0'),2),3)",
    generator = ['EPOSLHC', 'SIBYLL', 'QGSJET', 'Pythia8-Forward'],
    # generator = ['Pythia8-Forward'],
    energy = energy,
    nsample = 100,
)

In [14]:
model.add_production_2bodydecay(
    pid0 = "221",
    pid1 = "22",
    br = "2.*0.39 * coupling**2 * pow(1.-pow(mass/self.masses('pid0'),2),3)",
    generator = ['EPOSLHC', 'SIBYLL', 'QGSJET', 'Pythia8-Forward'],
    # generator = ['Pythia8-Forward'],
    energy = energy,
    nsample = 100, 
)

We can also produce the dark photon via its resonant mixing with the QCD vector mesons, in particular the $\omega$, $\rho$ and $\phi$ mesons. Following [1810.01879](https://arxiv.org/pdf/1810.01879.pdf), we can write

\begin{equation}
    \sigma(A') = \theta_V^2 \  \sigma(V)
    \quad\quad\text{with}\quad\quad
    \theta_V = \frac{\epsilon e}{g_V}  \frac{m_V^2}{m_{A'}^2 - m_V^2 + i m_V \Gamma_V}
\end{equation}

Here $g_\omega = 17$, $g_\rho=5$ and $g_\phi=-12.88$. In the following, we focus on the $\rho$, which provides the leading contribution. 

In [16]:
model.add_production_mixing(
    pid = "113",
    mixing = "coupling * 0.3/5. * self.masses('pid')**2/abs(mass**2-self.masses('pid')**2+self.masses('pid')*self.widths('pid')*1j)",
    generator = ['EPOSLHC', 'SIBYLL', 'QGSJET', 'Pythia8-Forward'],
    # generator = ['Pythia8-Forward'],
    energy = energy,
)

Dark photons can also be produced via dark Bremsstrahlung, so coherent radiation off a proton in processes such as $p p \to p p A'$. The spectra for LLPs have been exbtained following the description in [1708.09389](https://arxiv.org/abs/1708.09389) and are provided in the `files/direct` directory. As the FWW apprixmation loose validaty at high pT, we impose a cut on the pT of the dark photon and study the associated uncertainy by varing that cut as `0.5`, `1`, `2` GeV.


In [17]:
masses_brem = [ 
    0.01  ,  0.0126,  0.0158,  0.02  ,  0.0251,  0.0316,  0.0398,
    0.0501,  0.0631,  0.0794,  0.1   ,  0.1122,  0.1259,  0.1413,
    0.1585,  0.1778,  0.1995,  0.2239,  0.2512,  0.2818,  0.3162,
    0.3548,  0.3981,  0.4467,  0.5012,  0.5623,  0.6026,  0.631 ,
    0.6457,  0.6607,  0.6761,  0.6918,  0.7079,  0.7244,  0.7413,
    0.7586,  0.7762,  0.7943,  0.8128,  0.8318,  0.8511,  0.871 ,
    0.8913,  0.912 ,  0.9333,  0.955 ,  0.9772,  1.    ,  1.122 ,
    1.2589,  1.4125,  1.5849,  1.7783,  1.9953,  2.2387,  2.5119,
    2.8184,  3.1623,  3.9811,  5.0119,  6.3096,  7.9433, 10.    
]

model.add_production_direct(
    label = "Brem",
    energy = energy,
    condition = ["p.pt<1", "p.pt<2", "p.pt<0.5", "p.pt<1"],
    # condition = ["p.pt<1"],
    coupling_ref=1,
    masses = masses_brem,
)

obtained in [1810.01879](https://arxiv.org/pdf/1810.01879.pdf) using MadGraph plus Pythia, which are provided in the `files/direct` directory. 

In [18]:
#masses_dy = [1.5849, 1.7783, 1.9953,2.2387, 2.5119, 2.8184, 3.1623, 3.9811, 5.0119, 6.3096, 7.9433, 10.]
#model.add_production_direct(
#    label = "DY",
#    energy = energy,
#    coupling_ref=1,
#    masses = masses_dy,
#    condition='True',
#)

Initialize FORESEE

In [19]:
foresee_darkphoton = Foresee(path=src_path)
foresee_darkphoton.set_model(model=model)

### Save the A' spectrum to LLP_spectra folder

In the following, we will obtain the projected sensitivity for the LLP model. For this, we first define a grid of couplings and masses, and then produce the corresponding fluxes. 

In [21]:
masses=[round(x,5) for x in np.logspace(-2,0.2,22+1)]
couplings = np.logspace(-9,-3,121) 

for mass in masses: foresee_darkphoton.get_llp_spectrum(mass=mass,coupling=1)

saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_221_EPOSLHC_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_221_SIBYLL_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_221_QGSJET_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_221_Pythia8-Forward_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_111_EPOSLHC_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_111_SIBYLL_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_111_QGSJET_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_111_Pythia8-Forward_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_113_EPOSLHC_m_0.01.npy
saving file with mass =  0.01
save data to file: ./model/LLP_spectra/13.6TeV_113_SIBYLL_